In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from glob import glob

print(os.getcwd())
files = glob('./혼밥족_data/연도별 세대원수/*.csv')
print(files)

c:\Users\go121\practice_jupyter\개인공부용\과제\20240927
['./혼밥족_data/연도별 세대원수\\2014_세대원수별 세대수.csv', './혼밥족_data/연도별 세대원수\\2015_세대원수별 세대수.csv', './혼밥족_data/연도별 세대원수\\2016_세대원수별 세대수.csv', './혼밥족_data/연도별 세대원수\\2017_세대원수별 세대수.csv', './혼밥족_data/연도별 세대원수\\2018_세대원수별 세대수.csv']


In [8]:
for i, file in enumerate(files):
    df = pd.read_csv(file, encoding='cp949')
    
    #전처리
    df['행정구역코드'] = list(map(lambda x : re.findall('[0-9]+',x)[-1], df['행정구역']))
    new_df = df.drop(0)
    new_df=new_df[pd.to_numeric(new_df['행정구역코드'])%100000!=0]
    s_df=new_df[new_df['행정구역'].str.contains('서울특별시')]
    s_df = s_df.iloc[:,1:]
    s_df = s_df.set_index('행정구역코드').stack().reset_index()
    s_df['기준일']=list(map(lambda x: x.split('_')[0],s_df['level_1']))
    s_df['세대']=list(map(lambda x: x.split('_')[1],s_df['level_1']))
    s_df=s_df.drop('level_1',axis=1)
    s_df.columns=['행정구역코드', '값', '기준일', '세대']
    s_df['값']=pd.to_numeric(s_df['값'].str.replace(',',''))
    result_df=s_df.set_index(['행정구역코드','기준일','세대']).unstack().reset_index()
    result_df.columns=['행정구역코드', '기준일', '10인세대', '1인세대', '2인세대', '3인세대', '4인세대', '5인세대', '6인세대', '7인세대', '8인세대', '9인세대', '전체세대']
    result_df['5인이상세대']=result_df['5인세대']+result_df['6인세대']+result_df['7인세대']+result_df['8인세대']+result_df['9인세대']+result_df['10인세대']
    result_df=result_df[['행정구역코드','기준일','1인세대','2인세대','3인세대','4인세대','5인이상세대']]
    result_df['세대합계']=np.sum(result_df.iloc[:,2:].values,axis=1)
    result_df['1인세대비율']=round(result_df['1인세대']/result_df['세대합계'],ndigits=2)
    result_df['연도']=result_df['기준일'].str[:4]
    result_df['월']=result_df['기준일'].str[-3:-1]
    if i == 0:
        res = result_df.copy()
    else :
        res = pd.concat([res,result_df])
    
display(res)

,행정구역코드,기준일,1인세대,2인세대,3인세대,4인세대,5인이상세대,세대합계,1인세대비율,연도,월
0,1111051500,2014년01월,2076,1114,1083,1158,446,5877,0.35,2014,01
1,1111051500,2014년02월,2084,1113,1079,1168,443,5887,0.35,2014,02
2,1111051500,2014년03월,2083,1116,1088,1174,442,5903,0.35,2014,03
3,1111051500,2014년04월,2084,1114,1078,1169,431,5876,0.35,2014,04
4,1111051500,2014년05월,2086,1113,1079,1164,428,5870,0.36,2014,05
...,...,...,...,...,...,...,...,...,...,...,...
5083,1174070000,2018년08월,3350,2348,2366,2418,672,11154,0.30,2018,08
5084,1174070000,2018년09월,3366,2361,2354,2412,669,11162,0.30,2018,09
5085,1174070000,2018년10월,3379,2377,2340,2401,666,11163,0.30,2018,10
5086,1174070000,2018년11월,3392,2378,2339,2382,659,11150,0.30,2018,11
